# 线程和进程
## 线程
### 用threading.Thread 直接在线程中运行函数

In [18]:
import threading


def thrfun(x, y):
    for i in range(x, y):
        print("%d^2 = %d" % (i, i * i))


ta = threading.Thread(target=thrfun, args=(1, 10))
tb = threading.Thread(target=thrfun, args=(16, 26))
ta.start()
tb.start()

1^2 = 1
2^2 = 4
3^2 = 9
4^2 = 16
5^2 = 25
6^2 = 36
7^2 = 49
8^2 = 64
9^2 = 81
16^2 = 256
17^2 = 289
18^2 = 324
19^2 = 361
20^2 = 400
21^2 = 441
22^2 = 484
23^2 = 529
24^2 = 576
25^2 = 625


### 通过继承 threading.Thread类来创建线程

In [9]:
import threading


class myThread(threading.Thread):
    def __init__(self, mynum):
        super().__init__()
        self.mynum = mynum

    def run(self):
        for i in range(self.mynum, self.mynum + 5):
            print("%d^2 = %d" % (i, i * i))


ma = myThread(1)
mb = myThread(16)
ma.start()
mb.start()

1^2 = 1
2^2 = 4
3^2 = 9
4^2 = 16
5^2 = 25
16^2 = 256
17^2 = 289
18^2 = 324
19^2 = 361
20^2 = 400


### 线程类Thread使用   

- 方法  
    - join(\[timeout\]): 当某个线程或函数执行时需要等另一个线程完成操作后才能继续，则应调用另一个线程的joint()方法，其中的可选参数用于指定线程运行的最长时间。
        - isAlive()：方法用于查看线程是否运行。
- 属性
    - name：为线程设置线程名；
    - daemon：用来设置线程是否随主线程退出而退出，一般来说其属性值为True时不会随主线程退出而退出。

In [17]:
import threading
import time


def thrfun(x, y, thr=None):
    if thr:  #如果传入线程了参数，则执行join函数，表明等待传入的线程结束后运行
        thr.join()
    else:
        time.sleep(2)  #先休眠两秒
    for i in range(x, y):
        print("%d*%d=%d" % (i, i, i * i))


ta = threading.Thread(target=thrfun, args=(1, 6))
tb = threading.Thread(target=thrfun, args=(16, 21, ta))
ta.start()
tb.start()
#执行过程，ta先休眠两秒，tb运行时需要等待ta运行结束后在运行

1*1=1
2*2=4
3*3=9
4*4=16
5*5=25
16*16=256
17*17=289
18*18=324
19*19=361
20*20=400


In [22]:
#演示了deamon属性的作用
import threading
import time


class myThread(threading.Thread):
    def __init__(self, mynum):
        super().__init__()
        self.mynum = mynum

    def run(self):
        time.sleep(1)
        for i in range(self.mynum, self.mynum + 5):
            print("%d*%d=%d" % (i, i, i * i))


print('start...')
ma = myThread(1)
mb = myThread(16)
ma.daemon = True  #这里设置当主线程退出时，次线程也退出
mb.deamon = True
ma.start()
mb.start()
print('end...')

#在交互模式下运行实例，则还是会有输出。因为在交互模式的主线程只有在退出python时才终止。

start...
end...
1*1=116*16=256
2*2=4
3*3=9
17*17=289
18*18=324
19*19=361
20*20=400

4*4=16
5*5=25


### 线程的同步和通信
> Python中可以使用threading模块中的对象Lock和RLock(可重入锁)进行简单的线程同步。对于同一时刻只允许一个线程操作的数据对象，可以把操作过程放在Lock和RLock的acquire方法和release方法之间。RLock可以在同一调用链中有多次请求而不会锁死，Lock则会锁死。

In [28]:
import threading
import time


class myThread(threading.Thread):
    def run(self):
        global x
        lock.acquire()  #开始锁定
        for i in range(3):
            x += 10
        time.sleep(1)
        print(x)
        lock.release()  #释放锁


def main():
    thrs = []
    for item in range(5):  # 创建了5个线程
        thrs.append(myThread())
    for item in thrs:  # 开始运行
        item.start()


x = 0
lock = threading.RLock()  #创建一个锁
main()

30
60
90
120
150


- 线程间的同步还可以使用threading模块中的条件变量，队列等来进行。

- 以下演示通过threading模块的Event对象，实现线程间的同步

In [35]:
import threading
import time


class myThreada(threading.Thread):
    def run(self):
        evt.wait()  #等待标志为True
        print(self.name, ':Good morning!')
        evt.clear()  #将标志重置为FALSE
        time.sleep(1)
        evt.set()  #将标志置位
        time.sleep(1)
        evt.wait()
        print(self.name, ":I'm good.")


class myThreadb(threading.Thread):
    def run(self):
        print(self.name, ":Good morning!")
        evt.set()
        time.sleep(1)
        evt.wait()
        print(self.name, ':How are you?')
        evt.clear()
        time.sleep(1)
        evt.set()


evt = threading.Event()  #创建一个事件对象，管理标志


def main():
    John = myThreada()
    John.name = "John"

    Smith = myThreadb()
    Smith.name = 'Smith'

    John.start()
    Smith.start()


main()

Smith :Good morning!
John :Good morning!
Smith :How are you?
John :I'm good.


### 进程
- Python3对多进程支持的是multiprocessing模块和subprocess模块。

- subprocess模块可以用于创建新的进程，并获取它的输入，输出及错误信息。它提供了更高级的接口，可以替换os.system、os.spawn\*、popen等

In [1]:
import subprocess
# 创建新进程运行程序，输入和输出绑定到父进程，返回新进程退出码
print('call() test:', subprocess.call(['python', './src//hello.py']))

call() test: 0


In [2]:
# 创建新进程运行程序，输入和输出绑定到父进程，退出码为0正常返回，否则引发 calledprocesserror
print('call() test:', subprocess.check_call(['python', './src//hello.py']))

call() test: 0


In [3]:
#创建创建新进程运行程序，元组形式返回新进程退出码和输出
print('getstatusoutput() test:',
      subprocess.getstatusoutput(['python', './src//hello.py']))

getstatusoutput() test: (0, '0 Hello!\n1 Hello!\n2 Hello!\n3 Hello!\n4 Hello!\n5 Hello!\n6 Hello!\n7 Hello!\n8 Hello!\n9 Hello!')


In [29]:
# 创建新进程运行程序，返回新进程的输出(字符串)
print('getoutput() test:', subprocess.getoutput(['python', './src//hello.py']))

getoutput() test: 0 Hello!
1 Hello!
2 Hello!
3 Hello!
4 Hello!
5 Hello!
6 Hello!
7 Hello!
8 Hello!
9 Hello!


In [30]:
# 创建新进程运行程序，返回新进程的输出
print('check_output() test:',
      subprocess.check_output(['python', './src//hello.py']))

check_output() test: b'0 Hello!\r\n1 Hello!\r\n2 Hello!\r\n3 Hello!\r\n4 Hello!\r\n5 Hello!\r\n6 Hello!\r\n7 Hello!\r\n8 Hello!\r\n9 Hello!\r\n'


#### 用Popen类创建进程
> 上边提到的几个函数，其本质上都是通过Popen类来实现的简单版本的进程创建函数。直接使用Popen类不仅可以以新进程方式运行命令

In [42]:
import subprocess
prcs = subprocess.Popen(['python', './src//echo.py'],
                        stdout=subprocess.PIPE,
                        stdin=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        universal_newlines=True,
                        shell=True)
prcs.communicate('These strings are from stdin.')
print("subprocess pid:", prcs.pid)
print("\nSTDOUT:")
print(prcs.communicate()[0])
print("STDERR:")
print(prcs.communicate()[1])

subprocess pid: 11868

STDOUT:
echo 0
echo 1
echo 2
echo 3
echo 4
echo 5
echo 6
echo 7
echo 8
echo 9
These strings are from stdin. from echo.py

STDERR:
Traceback (most recent call last):
  File "./src//echo.py", line 8, in <module>
    pprint("error")
NameError: name 'pprint' is not defined



- 通过communicate()方法，可以为多个子进程之间建立“管道线”，即数据从一个进程输入并输出到另一个进程，这有点类似于“击鼓传花”

In [5]:
import subprocess
pythonFile =  './src//pipe.py'
with open(pythonFile) as fr:
    for line in fr.readlines():
        print(line)
print("---------------------")
processes = []
psum = 5
for i in range(psum):
    processes.append(
        subprocess.Popen(['python',pythonFile],
                         stdout=subprocess.PIPE,
                         stdin=subprocess.PIPE,
                         universal_newlines=True,
                         shell=True))

processes[0].communicate('0 bouquet of flowers!')
for before, after in zip(processes[:psum], processes[1:]):
    after.communicate(before.communicate()[0])
print('\nSum of Processes : %d' % psum)
print()
for item in processes:
    print(item.communicate()[0])

a = input()

a = a.split(' ')

a[0] = str(int(a[0])+1)

print(' '.join(a))
---------------------

Sum of Processes : 5

1 bouquet of flowers!

2 bouquet of flowers!

3 bouquet of flowers!

4 bouquet of flowers!

5 bouquet of flowers!

